# Sanity check
## 1. Cel sprawozdania
Celem tego pliku jest dokumentacja kodu oraz przebadanie poprawności naszej symulacji poprzez porównanie z teoretycznym, spodziewanym wynikiem. 

In [241]:
import matplotlib
matplotlib.use('TkAgg')  # or 'Qt5Agg'
import numpy as np
import simpy
import seaborn
import matplotlib.pyplot as plt

In [242]:
class Pacjent:
    def __init__(self, id):
        self.id = id
        self.arrival_time = None
        self.service_start_time = None
        self.service_end_time = None


In [243]:
class Gabinet:
    def __init__(self, id, env):
        self.id = id
        self.env = env
        self.patients_served = 0

# Parametry kliniki:
- **number_of_ rooms**: liczba gabinetów
- **service_time**: Jak długo obsługiwany jest pacjent [minuty]
- **lambda_per_hour**: Ile pacjentów przychodzi co godzinę (dzieli się przez 60 bo zamieniamy na minuty)
- **sim_time**: Długość symulacji

# Pomocnicze zmienne:
- **queue_array**: Kolejka (podgląd czekających pacjentów)
- **seed**: seed generatora liczb pseudolosowych

In [244]:
class Clinic:
    def __init__(self, env, number_of_rooms, service_time, queue_array=[], lambda_per_hour=12/60, seed=None, sim_time=480):
        self.curr_patient_id = 1
        self.env = env
        self.service_time = service_time
        self.lambda_per_hour = lambda_per_hour
        self.seed = seed
        self.list_rooms = [Gabinet(id=i + 1, env=self.env) for i in range(number_of_rooms)]
        self.rooms = simpy.Store(self.env, capacity=number_of_rooms)
        self.sim_time = sim_time
        self.queue_array = queue_array
        for room in self.list_rooms:
            self.rooms.put(room)
        self.processed_patients = []

    def czas(self, time_in_minutes):
        hours = 9+time_in_minutes//60
        minutes = time_in_minutes%60
        if len(str(int(minutes)))==1:
            return f"{int(hours)}:0{int(minutes)}"
        return f"{int(hours)}:{int(minutes)}"

    def time_between_new_patients(self):
        if self.seed:
            np.random.seed(self.seed)
        return np.random.exponential(1/self.lambda_per_hour)
    
    def generate_patients(self):
        while True:
            if self.env.now >= self.sim_time-self.service_time: # Pacjenci nie przychodzą przed {service_time} zamknięciem
                break
            patient = Pacjent(id=self.curr_patient_id)
            patient.arrival_time = self.env.now
            print(f"Czas {self.czas(np.trunc(self.env.now))}: Pacjent {patient.id} przybył do kliniki")
            self.env.process(self.serve_patient(patient))
            self.curr_patient_id += 1
            yield self.env.timeout(self.time_between_new_patients())

    def serve_patient(self, patient): 
        room = yield self.rooms.get()
        self.queue_array.append(patient.id)
        if self.env.now >= self.sim_time-self.service_time: # Nie zdążyło obsłużyć pacjenta przed zamknięciem
            print(f"Czas {self.czas(np.trunc(self.env.now))}: Pacjent {patient.id} - nie zdążył zostać obsłużony przed zamknięciem kliniki")
            yield self.rooms.put(room)
            return
        patient.service_start_time = self.env.now
        print(f"Czas {self.czas(np.trunc(self.env.now))}: Pacjent {patient.id} wchodzi do gabinetu {room.id} ")
        self.queue_array.remove(patient.id)
        yield self.env.timeout(self.service_time)

        patient.service_end_time = self.env.now
        room.patients_served += 1
        print(f"Czas {self.czas(np.trunc(self.env.now))}: Pacjent {patient.id} wychodzi z gabinetu {room.id}, czekal od {self.czas(np.trunc(patient.arrival_time))} do {self.czas(np.trunc(patient.service_start_time))}")
        self.processed_patients.append(patient)
        yield self.rooms.put(room)

    def run(self):
        self.env.process(self.generate_patients())
        env.run(until=self.sim_time)

    def stats(self):
        def patient_bar_plot():
            patients_served_ls = []
            id_ls = []
            for room in self.list_rooms:
                patients_served_ls.append(room.patients_served)
                id_ls.append(str(room.id))
            fig = seaborn.barplot(x=id_ls, y=patients_served_ls)
            fig.set_xlabel('Gabinet')
            fig.set_ylabel('Ilosc Pacjentow')
            plt.savefig("patients_served_noshow.png")
            plt.show()

        patient_bar_plot()

In [245]:
#bez umawiania
env = simpy.Environment()

In [246]:
clinic = Clinic(env, number_of_rooms=3, service_time=15)

In [247]:
clinic.run()

Czas 9:00: Pacjent 1 przybył do kliniki
Czas 9:00: Pacjent 1 wchodzi do gabinetu 1 
Czas 9:04: Pacjent 2 przybył do kliniki
Czas 9:04: Pacjent 2 wchodzi do gabinetu 2 
Czas 9:15: Pacjent 1 wychodzi z gabinetu 1, czekal od 9:00 do 9:00
Czas 9:19: Pacjent 2 wychodzi z gabinetu 2, czekal od 9:04 do 9:04
Czas 9:27: Pacjent 3 przybył do kliniki
Czas 9:27: Pacjent 3 wchodzi do gabinetu 3 
Czas 9:29: Pacjent 4 przybył do kliniki
Czas 9:29: Pacjent 4 wchodzi do gabinetu 1 
Czas 9:32: Pacjent 5 przybył do kliniki
Czas 9:32: Pacjent 5 wchodzi do gabinetu 2 
Czas 9:36: Pacjent 6 przybył do kliniki
Czas 9:42: Pacjent 3 wychodzi z gabinetu 3, czekal od 9:27 do 9:27
Czas 9:42: Pacjent 6 wchodzi do gabinetu 3 
Czas 9:44: Pacjent 4 wychodzi z gabinetu 1, czekal od 9:29 do 9:29
Czas 9:47: Pacjent 5 wychodzi z gabinetu 2, czekal od 9:32 do 9:32
Czas 9:57: Pacjent 7 przybył do kliniki
Czas 9:57: Pacjent 7 wchodzi do gabinetu 1 
Czas 9:57: Pacjent 6 wychodzi z gabinetu 3, czekal od 9:36 do 9:42
Czas 9:57:

In [248]:
clinic.queue_array

[86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98]

In [249]:
clinic.stats()